In [ ]:
import os
from fastbook import search_images_bing, download_url

api_key = os.environ.get("AZURE_API_KEY", 'XXX')
api_key

In [ ]:
from fastbook import Path, download_images, get_image_files

bear_types = ['grizzly', 'black', 'teddy']
path = Path('bears')

if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(api_key, f'{o} bear')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
import fastbook
fns = get_image_files(path)
failed = fastbook.verify_images(fns)
failed.map(Path.unlink)

In [ ]:
import fastbook

bears = fastbook.DataBlock(
    blocks = (fastbook.ImageBlock, fastbook.CategoryBlock),
    get_items=fastbook.get_image_files,
    splitter=fastbook.RandomSplitter(valid_pct=0.2, seed=42),
    get_y=fastbook.parent_label,
    item_tfms=fastbook.Resize(128, fastbook.ResizeMethod.Pad, pad_mode='zeros')
)

In [ ]:
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
bears = bears.new(item_tfms=fastbook.RandomResizedCrop(128, min_scale=0.3))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
bears = bears.new(item_tfms=fastbook.Resize(128), batch_tfms=fastbook.aug_transforms(mult=2))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
path = fastbook.Path('bears')
bears = bears.new(
    item_tfms=fastbook.RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=fastbook.aug_transforms())
dls = bears.dataloaders(path)


In [ ]:
learn = fastbook.vision_learner(dls, fastbook.resnet18, metrics=fastbook.error_rate)
learn.fine_tune(4)

In [ ]:
interp = fastbook.ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(5, nrows=1)

In [ ]:

from fastai.vision.widgets import ImageClassifierCleaner


cleaner = ImageClassifierCleaner(learn)
cleaner